<a href="https://colab.research.google.com/github/AntonioHenriqueAC/tcc/blob/master/google_colab/Barcode_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conectando o drive ao colab #

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Baixando as bibliotecas necessárias

In [2]:
!pip3 install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.2MB 48kB/s 
     |████████████████████████████████| 368kB 33.0MB/s 
     |████████████████████████████████| 3.2MB 36.1MB/s 
ERROR: tensorflow 2.2.0rc2 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0


In [3]:
!pip3 install imageai --upgrade

     |████████████████████████████████| 184kB 1.4MB/s 


# Treinar novo Modelo !

## 1. Baixando a CNN pré-treinada ##


In [0]:
!wget https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5

## 2. Carregando os arquivos do drive no colab ##

In [0]:
!unzip "/content/drive/My Drive/Colab/datasets/barcode4.zip" 

## 3. Treinando o modelo ##

In [0]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="barcode")
trainer.setTrainConfig(object_names_array=["barcode"], batch_size=4, num_experiments=11, train_from_pretrained_model="pretrained-yolov3.h5")
trainer.trainModel()

## 4. Avaliando os modelos gerados ##

In [0]:
metrics = trainer.evaluateModel(model_path="barcode/models", json_path="barcode/json/detection_config.json", iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)

# Utilizar modelo já treinado.

## 1. Criando as pastas ##

In [0]:
import os

path, dirs, files = next(os.walk("/content/drive/My Drive/Colab/Corridas"))
file_count_corridas = len(dirs)
print("Número de corridas na pasta: ", file_count_corridas - 1)


newFolderCorrida = str(file_count_corridas)
folderCorrida = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida
print("..")
print("..")
print("..")
!mkdir $folderCorrida
print("Criando pastas Corrida_"+newFolderCorrida)

folderTagsJson = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida + "/tags_json"
print("..")
print("..")
print("..")
try:
  !mkdir $folderTagsJson
  print("Criando pastas /tags_json")
except:
  print("Já existe a pasta /tags_json")

folderTagsDetected = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida + "/tags_detected"
print("..")
print("..")
print("..")
try:
  !mkdir $folderTagsDetected
  print("Criando pastas /tags_detected")
except:
  print("Já existe a pasta /tags_detected")

print("..")
print("..")
print("..")
try:
  !mkdir barcode-detected-objects
  print("Criando pastas /barcode-detected-objects")
except:
  print("Já existe a pasta /barcode-detected-objects")

## 2. Setando as variaveis ##


In [0]:
modelPath = "/content/drive/My Drive/Colab/models/barcode/detection_model-ex-007--loss-0003.066.h5"
JsonPath = "drive/My Drive/Colab/json/detection_config_barcode.json"
InputImage = "drive/My Drive/Colab/Imagens_Originais/img10.jpeg"

## 3. Treinando modelo e recortando os codigos de barras da imagem original ##


In [0]:
from imageai.Detection.Custom import CustomObjectDetection

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(modelPath)
detector.setJsonPath(JsonPath)
detector.loadModel()

detections = detector.detectObjectsFromImage(input_image = InputImage, 
                                                           output_image_path="barcode-detected-OriginalBoxes.jpg",
                                                           minimum_percentage_probability=25)

def boxPoints(e):
  return e['box_points']  
detections.sort(key=boxPoints)

for e in detections:
  e['box_points'][0] = e['box_points'][0] - 28 ## x1
  e['box_points'][1] = e['box_points'][1] - 55 ## y1
  e['box_points'][2] = e['box_points'][2] + 30 ## x2
  e['box_points'][3] = e['box_points'][3] + 50 ## y2

print("Número       " , ": ", "Porcentagem ", "      :    ", "box_points")
print("----------------------------------------------------------------")

print("")

for detection in detections:
    print(detection["name"], "      : ", detection["percentage_probability"], " : ", detection["box_points"])


print("----------------------------------------------------------------")
print("Quantidade de código de barras detectados na imagem: ", len(detections))

## 4. Identificando os Bounding Box e recortandando as imagens. 

In [0]:
%matplotlib inline
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import skimage.segmentation
import cv2

img = skimage.io.imread(InputImage)
bbox = []

for e in detections:
    bbox.append(e['box_points'])

for i, (x1,y1,x2,y2) in enumerate(bbox):

    if i in {0,1,2,3,4,5,6,7,8,9}:
      i = '00' + str(i)
    
    if(int(i) > 9 and int(i) < 100):
      i = '0' + str(i)

    img2 = cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 0)
    out = img[y1:y2, x1:x2]

    cv2.imwrite('barcode-detected-objects/barcode-00'+str(i)+'.jpg', out)

cv2.imwrite('barcode-detected-New.jpg', img)

print("Boxes recortados com sucesso!")

**Visualizando nova a imagem gerada a partir das novas Bounding Boxes.**

In [0]:
from google.colab.patches import cv2_imshow
cv2_imshow(img)

## 4. Movendo as pastas para o diretório correto. ##


In [0]:
!mv /content/barcode-detected-OriginalBoxes.jpg $folderCorrida
!mv /content/barcode-detected-New.jpg $folderCorrida
!mv /content/barcode-detected-objects $folderCorrida

barcode = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida + "/barcode-detected-objects"
images = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida + "/tags_images"
!mv  $barcode $images

print("Pastas movidas para o drive com sucesso!")